# Generating object masks from input prompts with SAM



## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
# !pip install segment-geospatial leafmap localtileserver

In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff

## Create an interactive map

In [2]:
m = leafmap.Map(center=[10.029939778759502, 105.76756428014754], zoom=17, height="600px", width="800px")
m.add_basemap("SATELLITE")
m

Map(center=[10.029939778759502, 105.76756428014754], controls=(ZoomControl(options=['position', 'zoom_in_text'…

ImportError: Please install mss using 'pip install mss'

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [5]:
if m.user_roi is not None:
    bbox = m.user_roi_bounds()
    bbox
else:
    bbox = [105.764, 10.0274, 105.7715, 10.0329]

In [ ]:
bbox

In [6]:
image = "ctu.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=16, source="Satellite", overwrite=True)

Downloaded image 1/6
Downloaded image 2/6
Downloaded image 3/6
Downloaded image 4/6
Downloaded image 5/6
Downloaded image 6/6
Saving GeoTIFF. Please wait...
Image saved to ctu.tif


You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# import geemap 
# geemap.update_package() 
# !pip install google.colab
# !pip install leafmap

Display the downloaded image on the map.

In [7]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=15837861.0, center=[10.02978393817378, 105.76869606971742], controls=(ZoomControl(options=['positio…

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

Set `automatic=False` to disable the `SamAutomaticMaskGenerator` and enable the `SamPredictor`.

In [8]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    automatic=False,
    sam_kwargs=None,
)

Specify the image to segment. 

In [9]:
sam.set_image(image)

## Image segmentation with input points

A single point can be used to segment an object. The point can be specified as a tuple of (x, y), such as (col, row) or (lon, lat). The points can also be specified as a file path to a vector dataset. For non (col, row) input points, specify the `point_crs` parameter, which will automatically transform the points to the image column and row coordinates.

Try a single point input:

In [10]:
point_coords = [[105.76755698844329, 10.029964842544715]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="mask1.tif")
m.add_raster("mask1.tif", layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)
m

Map(bottom=15837826.0, center=[10.030149999999992, 105.76774999999999], controls=(ZoomControl(options=['positi…

Try multiple points input:

In [11]:
point_coords = [[105.76755698844329, 10.029964842544715], [105.76807137023872, 10.02981441080665], [105.76721100868511, 10.029391083681455]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output="mask2.tif")
m.add_raster("mask2.tif", layer_name="Mask2", nodata=0, cmap="Greens", opacity=1)
m

Map(bottom=7919005.0, center=[10.031379229773162, 105.76388955116273], controls=(ZoomControl(options=['positio…

## Interactive segmentation

Display the interactive map and use the marker tool to draw points on the map. Then click on the `Segment` button to segment the objects. The results will be added to the map automatically. Click on the `Reset` button to clear the points and the results.

In [12]:
m = sam.show_map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…